# Day 4: Secure Container

https://adventofcode.com/2019/day/4

The Venus fuel depot is protected by a password. 

The password was written on a sticky note, but someone threw it out.

However, a few key facts about the password are known:

- It is a six-digit number.
- The value is within the range given in your puzzle input.
- Two adjacent digits are the same (like 22 in 122345).
- Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).

## Intelligent bruteforcing

We are given a set of rules that substantially reduce the search space.
    
### The password is a 6 digits number. 

With this constraint, the search space is then reduced to 1,000,000 possibilities.

_We will work with individual digits, which will make things easier, when applying other constraints._

In [1]:
module Number =

    type Number = int*int*int*int*int*int

    let ofInt (i:int) : Number =
        i |> (string) |> Seq.map ((string) >> Int32.Parse) |> List.ofSeq |> function | [a;b;c;d;e;f] -> (a,b,c,d,e,f) | _ -> failwith "Invalid Number"

123456 |> Number.ofInt |> (string)

(1, 2, 3, 4, 5, 6)

### The password is a number within a given range

This will give as a min and max bound for our enumeration of possible values.

_This constraint is the input._

In [2]:
open Number

module Number = 

    let asInt ((a,b,c,d,e,f):Number) : int =
        a*100000+b*10000+c*1000+d*100+e*10+f

    let greaterThan n2 n1 =
        (asInt n1) > (asInt n2)

    let lowerThan n2 n1 =
        (asInt n1) < (asInt n2)


Number.greaterThan (0,0,0,1,2,3) (0,0,0,1,2,4)

True

### The password is a sequence of increasing digits

By applying this constraint, we can skip many numbers.

We will work with digits separately, waiting for the edge condition:

> any number of consecutive nines
 
Which we will handle performing:
 
> increment previous digit, set the following to the same value

_Since they must not be lower value_

Given a starting value, we will keep computing the next lowest possible number.

As an example, when given 123**499**, the next acceptable value should be 123**555**

In [3]:
let nextNumber (n:Number) : Number option = 
    match n with
    | (9,9,9,9,9,9) -> None
    | (a,9,9,9,9,9) -> Some(a+1,a+1,a+1,a+1,a+1,a+1)
    | (a,b,9,9,9,9) -> Some(a  ,b+1,b+1,b+1,b+1,b+1)
    | (a,b,c,9,9,9) -> Some(a  ,b  ,c+1,c+1,c+1,c+1)
    | (a,b,c,d,9,9) -> Some(a  ,b  ,c  ,d+1,d+1,d+1)
    | (a,b,c,d,e,9) -> Some(a  ,b  ,c  ,d  ,e+1,e+1)
    | (a,b,c,d,e,f) -> Some(a  ,b  ,c  ,d  ,e  ,f+1)

[ 
    (0,0,0,0,0,0);
    (0,0,0,0,1,1);
    (0,0,0,0,1,9);
    (1,2,3,4,9,9);
    (9,9,9,9,9,9);
] |> List.map (fun n -> match (nextNumber n) with | Some next -> (n, (string) next) | None -> (n, "Out of range"))

index,Item1,Item2
0,"( 0, 0, 0, 0, 0, 0 )","(0, 0, 0, 0, 0, 1)"
1,"( 0, 0, 0, 0, 1, 1 )","(0, 0, 0, 0, 1, 2)"
2,"( 0, 0, 0, 0, 1, 9 )","(0, 0, 0, 0, 2, 2)"
3,"( 1, 2, 3, 4, 9, 9 )","(1, 2, 3, 5, 5, 5)"
4,"( 9, 9, 9, 9, 9, 9 )",Out of range


We can now implement a password generator that implement this constraint.

In [4]:
let increasingDigitsNumbers = seq { 
    yield! (0,0,0,0,0,0) |> Seq.unfold (
        // `unfold` requires returning (current state, next state) option
        fun n -> n |> nextNumber |> Option.map (fun next -> (n,next))
    )
}

increasingDigitsNumbers
|> Seq.take 10
|> Seq.map (string)

index,value
0,"(0, 0, 0, 0, 0, 0)"
1,"(0, 0, 0, 0, 0, 1)"
2,"(0, 0, 0, 0, 0, 2)"
3,"(0, 0, 0, 0, 0, 3)"
4,"(0, 0, 0, 0, 0, 4)"
5,"(0, 0, 0, 0, 0, 5)"
6,"(0, 0, 0, 0, 0, 6)"
7,"(0, 0, 0, 0, 0, 7)"
8,"(0, 0, 0, 0, 0, 8)"
9,"(0, 0, 0, 0, 0, 9)"


Let's verify those generated numbers

In [5]:
let isIncreasingDigitsNumber ((a,b,c,d,e,f):Number) =
    a<=b && b<=c && c<=d && d<=e && e<=f
    
increasingDigitsNumbers
|> Seq.filter ((not) << isIncreasingDigitsNumber)
|> Seq.length

0

### Password contains at least two adjacent, equal digits

This is pretty straight forward.

In [6]:
let hasAtLeastTwoAdjacentEqualDigits =
    function
    | (a,b,_,_,_,_) when a = b -> true
    | (_,b,c,_,_,_) when b = c -> true
    | (_,_,c,d,_,_) when c = d -> true
    | (_,_,_,d,e,_) when d = e -> true
    | (_,_,_,_,e,f) when e = f -> true
    | _ -> false
    
[ 
    (0,0,0,0,0,0);
    (0,0,0,0,1,1);
    (1,2,3,4,5,6);
    (1,2,3,4,9,9);
] |> List.map (fun n -> (n, hasAtLeastTwoAdjacentEqualDigits n))

index,Item1,Item2
0,"( 0, 0, 0, 0, 0, 0 )",True
1,"( 0, 0, 0, 0, 1, 1 )",True
2,"( 1, 2, 3, 4, 5, 6 )",False
3,"( 1, 2, 3, 4, 9, 9 )",True


### Test for a valid number

In [7]:
let validPassword (n:Number) =
    (hasAtLeastTwoAdjacentEqualDigits n) && (isIncreasingDigitsNumber n)

## Putting it all together

We can now generate all possible passwords with the given contraints

In [8]:
let acceptablePasswordsInRange min max =
    increasingDigitsNumbers
    |> Seq.filter ((not) << (Number.lowerThan min))
    |> Seq.filter ((not) << (Number.greaterThan max))
    |> Seq.filter (hasAtLeastTwoAdjacentEqualDigits)

acceptablePasswordsInRange (0,0,0,0,0,0) (1,1,1,1,1,1) |> Seq.rev |> Seq.take 4 |> Seq.map (string)

index,value
0,"(1, 1, 1, 1, 1, 1)"
1,"(0, 9, 9, 9, 9, 9)"
2,"(0, 8, 9, 9, 9, 9)"
3,"(0, 8, 8, 9, 9, 9)"


## Test cases

`111111` meets these criteria (double `11`, never decreases).

In [9]:
111111 |> Number.ofInt |> validPassword

True

`223450` does not meet these criteria (decreasing pair of digits `50`).

In [10]:
223450 |> Number.ofInt |> validPassword

False

`123789` does not meet these criteria (no double).

In [11]:
123789 |> Number.ofInt |> validPassword

False

## Problem

How many different passwords within the range given in your puzzle input meet these criteria?

Your puzzle input is `197487`-`673251`.

In [12]:
#r "nuget:Expecto"
    
open Expecto

try 
    let result = acceptablePasswordsInRange (197487 |> Number.ofInt) (673251 |> Number.ofInt) |> Seq.length
    let expected = 1640
    Expect.equal result expected (sprintf "Unexpected result: %A (Should be %A)" result expected)
    "That's the right answer! You are one gold star closer to rescuing Santa."
with
    | ex -> ex.ToString()

Installing package Expecto.done!

That's the right answer! You are one gold star closer to rescuing Santa.